In [ ]:
import math
import jax
import jax.numpy as jnp
import equinox as eqx
import optax
import matplotlib.pyplot as plt

In [ ]:
t = jnp.linspace(0.0, 1.0, 120)
y = jnp.stack((jnp.linspace(1.0, 3.0, 50), jnp.linspace(4.0, 6.0, 50))).reshape(-1, 1)
x = jnp.sin(2.0 * math.pi * jnp.matmul(y, t.reshape((1, -1)))).reshape(-1, t.shape[0], 1)

In [ ]:
class RNN(eqx.Module):
    hidden_size: int
    cell: eqx.nn.LSTMCell
    linear: eqx.nn.Linear

    def __init__(self, input_size, hidden_size, out_size, key):
        ckey, lkey = jax.random.split(key)
        self.hidden_size = hidden_size
        self.cell = eqx.nn.LSTMCell(
            input_size=input_size, hidden_size=hidden_size, key=ckey
        )
        self.linear = eqx.nn.Linear(hidden_size, out_size, key=lkey)

    def __call__(self, x):
        hidden_cell = (jnp.zeros(self.hidden_size), jnp.zeros(self.hidden_size))
        def fn(hid_cell, inp):
            return self.cell(inp, hid_cell), None

        (h, _), _ = jax.lax.scan(fn, hidden_cell, x)
        return self.linear(h)

In [ ]:
plt.plot(t, x[0])
plt.plot(t, x[-1])

In [ ]:
steps = 5000
hidden_size = 12

model = RNN(input_size=1, hidden_size=hidden_size, out_size=1, key=jax.random.PRNGKey(42))
print(model)
optim = optax.adam(1e-2)
opt_state = optim.init(eqx.filter(model, eqx.is_array))

@eqx.filter_value_and_grad
def loss_fn(model, x, y):
    pred = jax.vmap(model)(x)
    return optax.losses.squared_error(pred, y).mean()

@eqx.filter_jit
def evolve(model, x, y, opt_state):
    loss, grads = loss_fn(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    return model, loss, opt_state

for step in range(steps):
    model, loss, opt_state = evolve(model, x, y, opt_state)
    loss = loss.item()
    print(f"step={step}, loss={loss}")

In [ ]:
y_test = jnp.linspace(3.0, 4.0, 50).reshape(-1, 1)
x_test = jnp.sin(2.0 * math.pi * jnp.matmul(y_test, t.reshape((1, -1)))).reshape(-1, t.shape[0], 1)

loss_val, _ = loss_fn(model, x_test, y_test)
print(f"validation loss={loss_val}")